In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pip install cudf

In [ ]:
import pandas as pd
import cudf
import time
print(f"Pandas: {pd.__version__}, Cudf: {cudf.__version__}")

In [ ]:
#LOADING
starttime = time.time()
df_pandas = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
time_pandas = time.time() - starttime
print(f"Time to load: {time_pandas:.2f}s")

print(df_pandas.shape)
df_pandas.head()

In [ ]:
#LOADING
starttime = time.time()
df_cudf = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
time_cudf = time.time() - starttime
print(f"Time to load: {time_cudf:.2f}s")

print(df_cudf.shape)
df_cudf.head()

In [ ]:
#GROUPBY AND AGGREGATION
starttime = time.time()
agg_pandas = df_pandas.groupby('customer_id')[['price', 'sales_channel_id']].agg(['mean', 'min', 'max', 'std'])
time_pandas = time.time() - starttime
print(f"Time to aggregate: {time_pandas:.2f}s")

In [ ]:
#GROUPBY AND AGGREGATION
starttime = time.time()
agg_cudf = df_cudf.groupby('customer_id')[['price', 'sales_channel_id']].agg(['mean', 'min', 'max', 'std'])
time_cudf = time.time() - starttime
print(f"Time to aggregate: {time_cudf:.2f}s")

In [ ]:
#SORTING
starttime = time.time()
df_pandas = df_pandas.sort_values('t_dat', ignore_index=True)
time_pandas = time.time() - starttime
print(f"Time to sort: {time_pandas:.2f}s")

In [ ]:
#SORTING
starttime = time.time()
df_cudf = df_cudf.sort_values('t_dat', ignore_index=True)
time_cudf = time.time() - starttime
print(f"Time to sort: {time_cudf:.2f}s")

In [ ]:
#GAUSSRANK
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from scipy.special import erfinv as sp_erfinv
from cupyx.scipy.special import erfinv

In [ ]:
#GAUSSRANK
def gaussrank_cpu(data, epsilon = 1e-6):
    r_cpu = data.argsort().argsort()
    r_cpu = (r_cpu/r_cpu.max()-0.5)*2 # scale to (-1,1)
    r_cpu = np.clip(r_cpu,-1+epsilon,1-epsilon)
    r_cpu = sp_erfinv(r_cpu)
    return(r_cpu)

def gaussrank_gpu(data, epsilon = 1e-6):
    r_gpu = data.argsort().argsort()
    r_gpu = (r_gpu/r_gpu.max()-0.5)*2 # scale to (-1,1)
    r_gpu = cp.clip(r_gpu,-1+epsilon,1-epsilon)
    r_gpu = erfinv(r_gpu)
    return(r_gpu)

In [ ]:
#GAUSSRANK
data_cpu = df_pandas['price']
data_gpu = df_cudf['price'].values

In [ ]:
%%time 

gaussrank_cpu(data_cpu)

In [ ]:
%%time 

gaussrank_gpu(data_gpu)